<a href="https://colab.research.google.com/github/tanishq150802/ML7-RockPaperScissorGAME/blob/main/rps_bd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
!pip install tensorflow_addons
import tensorflow_addons as tfa
import numpy as np
import random
import matplotlib.pyplot as plt
!pip install pygame
import pygame as pg
import cv2 as cv
from google.colab.patches import cv2_imshow
import sklearn

     |████████████████████████████████| 686kB 8.3MB/s 
     |████████████████████████████████| 11.8MB 198kB/s 
pygame 2.0.1 (SDL 2.0.14, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19sum',
 'crema_d',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'dementiabank',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 

In [ ]:
#bhavya

(ds_train, ds_valid, ds_test), ds_info = tfds.load(
    'rock_paper_scissors',
    split=['train[:80%]', 'train[80%:]', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete1Y9POR/rock_paper_scissors-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete1Y9POR/rock_paper_scissors-test.tfrecord


Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(300,300, input_shape = (300,300,3)),                                       
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
])

In [ ]:
#Creating model

num_classes = 3
img_height = 300
img_width = 300
model = tf.keras.models.Sequential([
                    data_augmentation,
                    tf.keras.layers.Conv2D(32, 3, padding='same',  activation='relu',kernel_initializer='he_uniform',),
                    tf.keras.layers.MaxPooling2D(2),
                    tf.keras.layers.Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation='relu'),
                    tf.keras.layers.MaxPooling2D(2),
                    tf.keras.layers.Conv2D(64, 3, padding='same', kernel_initializer='he_uniform',activation='relu'),
                    tf.keras.layers.MaxPooling2D(2),
                    tf.keras.layers.Conv2D(64, 3, padding='same', kernel_initializer='he_uniform',activation='relu'),
                    tf.keras.layers.MaxPooling2D(2),
                    tf.keras.layers.Conv2D(128, 3, padding='same', kernel_initializer='he_uniform',activation='relu'),
                    tf.keras.layers.MaxPooling2D(2),
                    tf.keras.layers.Flatten(),
                    tf.keras.layers.Dense(128, kernel_initializer='he_uniform',activation = 'relu'),
                    tf.keras.layers.Dense(3, activation = 'softmax'),
                    #tf.keras.layers.Dense(3),
                    ])
# model = tf.keras.models.Sequential([
#   tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
#   tf.keras.layers.Conv2D(16, 3, activation='relu', padding="same"),
#   tf.keras.layers.MaxPooling2D(strides=(1,1)),
#   tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
#   tf.keras.layers.MaxPooling2D(strides=(1,1)),
#   tf.keras.layers.Conv2D(64, 3, activation='relu', padding="same"),
#   tf.keras.layers.MaxPooling2D(strides=(1,1)),
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dense(num_classes)])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 300, 300, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 64)       

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_valid = ds_valid.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_valid = ds_valid.cache()
ds_valid = ds_valid.shuffle(ds_info.splits['train'].num_examples)
ds_valid = ds_valid.batch(32)
ds_valid = ds_valid.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.cache()
ds_test = ds_test.shuffle(ds_info.splits['train'].num_examples)
ds_test = ds_test.batch(32)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(ds_train, epochs=3, validation_data=ds_valid)

Epoch 1/3


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


In [ ]:
test_loss, test_acc = model.evaluate(ds_test)

print('\nTest accuracy:', test_acc)

12/12 [==============================] - 1s 28ms/step - loss: 0.4394 - accuracy: 0.9059

Test accuracy: 0.9059139490127563


In [ ]:
#Getting image feed

path = "/content/Rock_paper_Seassors_1.png"
image = cv.imread(path)
# Loading the image

stretch_near = cv.resize(image,(300, 300),interpolation = cv.INTER_NEAREST)

In [ ]:
stretch_near.shape
stretch_near = stretch_near[np.newaxis, :]
stretch_near.shape

(1, 300, 300, 3)

In [ ]:
#Evaluating the move
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(stretch_near)
print(np.argmax(predictions), predictions[0])


0 [0.57611686 0.21194157 0.21194157]


In [ ]:
if np.argmax(predictions) == 0:
  print("rock")
elif np.argmax(predictions) == 1:
  print("paper")
elif np.argmax(predictions) == 2:
  print("scissor")

rock


In [ ]:
#Calculating F1 Score

sklearn.metrics.f1_score()

AttributeError: ignored

In [ ]:
type(float(predictions[0][0]))

float

In [ ]:
["rock", "paper", "scissor"]

['rock', 'paper', 'scissor']